In [11]:
# from google.colab import drive
# drive.mount('/content/drive')


In [12]:
# Directory paths

train_dir = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/train'
valid_dir = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/train'
test_dir = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/test'

# Step 1: Preprocessing the Data

In [13]:
import matplotlib
matplotlib.use("Agg")

# import tensorflow as tf
# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.optimizers import Adam
from keras.layers import Input
from keras.applications import VGG19
from keras.layers import Flatten, Dense, Dropout
# from keras.preprocessing import image
# import numpy as np
# import os
# from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [14]:
CLASSES = 6       # There are 6 classes of defects in each directory: Crazing, Inclusion, Patches, Pitted, Rolled, Scratches
HEIGHT = 224      # Height of the input image for the model
WIDTH = 224       # Width of the input image for the model
CHANNELS = 3      # Number of color channels in the image (RGB - Red, Green, Blue)
BATCH_SIZE = 16   # Number of images processed in a single batch during training or inference
Epochs = 5


In [15]:
# Create a VGG19 model pre-trained on ImageNet without including the top classification layers
baseModel = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(WIDTH, HEIGHT, CHANNELS)))

# Set all layers in the base VGG19 model to non-trainable (freeze the weights)
for layer in baseModel.layers:
    layer.trainable = False

# Create a Sequential model to stack layers on top of the pre-trained VGG19 base model
model = Sequential()

# Add the pre-trained VGG19 base model to the sequential model
model.add(baseModel)

# Flatten the output from the base model before passing it to fully connected layers
model.add(Flatten(name="flatten"))

# Add a fully connected dense layer with 1024 units and ReLU activation function
model.add(Dense(1024, activation='relu'))

# Apply dropout regularization to the previous layer to prevent overfitting (dropout rate: 0.5)
model.add(Dropout(0.5))

# Add another fully connected dense layer with 256 units and ReLU activation function
model.add(Dense(256, activation='relu'))

# Apply dropout regularization to the previous layer (dropout rate: 0.5)
model.add(Dropout(0.5))

# Add the output layer with 'CLASSES' number of units and softmax activation for multi-class classification
model.add(Dense(CLASSES, activation='softmax'))


In [16]:
# Display a summary of the neural network model architecture, showing the layer names, output shapes,
# and the number of parameters in each layer
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 1024)              25691136  
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               262400    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 6)                

In [17]:

# Data augmentation and normalization for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to [0,1]
    rotation_range=20,           # Randomly rotate images within 20 degrees
    shear_range=0.2,             # Apply shear transformation
    zoom_range=0.2,              # Apply random zoom
    horizontal_flip=True,        # Flip images horizontally
    vertical_flip=True           # Flip images vertically
)

# Data augmentation and normalization for the validation dataset
valid_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to [0,1]
    rotation_range=20,           # Randomly rotate images within 20 degrees
    shear_range=0.2,             # Apply shear transformation
    zoom_range=0.2,              # Apply random zoom
    horizontal_flip=True,        # Flip images horizontally
    vertical_flip=True           # Flip images vertically
)

# Normalization for the test dataset (no data augmentation for test data)
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0,1] for test data


In [18]:
# Generating batches of preprocessed images and their labels using flow_from_directory

# Training set generator
training_set = train_datagen.flow_from_directory(
    train_dir,                      # Directory containing training images
    target_size=(WIDTH, HEIGHT),    # Resizes images to match model input dimensions
    batch_size=BATCH_SIZE,          # Size of the batches of data (number of samples per gradient update)
    class_mode='categorical'        # Uses categorical labels for multi-class classification
)

# Validation set generator
validation_set = valid_datagen.flow_from_directory(
    valid_dir,                      # Directory containing validation images
    target_size=(WIDTH, HEIGHT),    # Resizes images to match model input dimensions
    batch_size=BATCH_SIZE,          # Size of the batches of data (number of samples per gradient update)
    class_mode='categorical'        # Uses categorical labels for multi-class classification
)

# Test set generator
test_set = test_datagen.flow_from_directory(
    test_dir,                       # Directory containing test images
    target_size=(WIDTH, HEIGHT),    # Resizes images to match model input dimensions
    batch_size=BATCH_SIZE,          # Size of the batches of data (number of samples per gradient update)
    class_mode='categorical',       # Uses categorical labels for multi-class classification
    shuffle=False                   # Disables shuffling to maintain order in the test set
)


Found 1656 images belonging to 6 classes.
Found 1656 images belonging to 6 classes.
Found 72 images belonging to 6 classes.


In [19]:
# Compiling the model with specified loss function, optimizer, and evaluation metrics

model.compile(
    loss="categorical_crossentropy",     # Defines the loss function for categorical classification problems
    optimizer=Adam(learning_rate=0.001), # Uses Adam optimizer with a learning rate of 0.001 for model optimization
    metrics=["accuracy"]                 # Evaluates model performance using accuracy metric
)


In [20]:
# Training the model using the fit method with training and validation data

history = model.fit(
    training_set,                                  # Training dataset
    epochs=Epochs,                                 # Number of training epochs
    steps_per_epoch=training_set.samples // BATCH_SIZE,  # Number of steps (batches) per epoch for the training set
    validation_data=validation_set,                # Validation dataset
    validation_steps=validation_set.samples // BATCH_SIZE  # Number of steps (batches) per epoch for the validation set
)


Epoch 1/5
103/103 [==============================] - 802s 8s/step - loss: 3.2886 - accuracy: 0.3841 - val_loss: 0.7420 - val_accuracy: 0.7955
Epoch 2/5
103/103 [==============================] - 778s 8s/step - loss: 0.9392 - accuracy: 0.6445 - val_loss: 0.4681 - val_accuracy: 0.8641
Epoch 3/5
103/103 [==============================] - 776s 8s/step - loss: 0.7084 - accuracy: 0.7384 - val_loss: 0.4364 - val_accuracy: 0.8501
Epoch 4/5
103/103 [==============================] - 798s 8s/step - loss: 0.6382 - accuracy: 0.7665 - val_loss: 0.2923 - val_accuracy: 0.9248
Epoch 5/5
103/103 [==============================] - 822s 8s/step - loss: 0.5676 - accuracy: 0.7884 - val_loss: 0.2993 - val_accuracy: 0.9150


In [21]:
# Evaluating the trained model using the test dataset

scores = model.evaluate(test_set)  # Evaluates the model's performance on the test set


5/5 [==============================] - 17s 3s/step - loss: 0.3296 - accuracy: 0.9306


In [22]:
# Printing the test loss value

print('Test Loss: ', scores[0])  # Prints the computed test loss value


Test Loss:  0.32958194613456726


In [23]:
# Printing the test accuracy

print('Test Accuracy: ', scores[1])  # Prints the computed test accuracy


Test Accuracy:  0.9305555820465088


In [24]:
# Saving the trained model

model.save('defect_detection_model.h5')  # Saves the trained model to a file named 'defect_detection_model.h5'


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
# Function to load an image using PIL library

from PIL import Image

def load_image(image_path): # Importing the Image module from the PIL library

    """
    Loads an image using the PIL library.
    
    Args:
    - image_path: The file path of the image to be loaded
    
    Returns:
    - img: The loaded image object
    """

    img = Image.open(image_path)
    return img


In [26]:
from PIL import Image
from keras.preprocessing.image import img_to_array

def preprocess_image(img):
    # Resize the image to the required input size (224x224 for VGG16)
    img = img.resize((224, 224))

    # Convert the image to RGB if it has a single channel (grayscale)
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Convert PIL Image to numpy array
    img_array = img_to_array(img)

    # Normalize pixel values to be in the range [0, 1]
    img_array = img_array / 255.0

    return img_array


In [27]:
from keras.models import load_model
import numpy as np

# Load the saved model
loaded_model = load_model('defect_detection_model.h5')


# Use the loaded model to make predictions on new data
def detect_defect(image_path):
    img = load_image(image_path)  # Load the image (implement this function)
    img = preprocess_image(img)   # Preprocess the image (implement this function)

    # Make predictions using the loaded model
    prediction = loaded_model.predict(np.expand_dims(img, axis=0))

    # Check the prediction result
    if prediction[0][0] >= 0.5:  # Assuming binary classification (defect or non-defect)
        return "Defect Detected"
    else:
        return "No Defect Detected"


In [ ]:
# ! pip install opencv-python-headless

In [ ]:
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import cv2  # Import OpenCV for image manipulation

# Load the saved model
loaded_model = load_model('defect_detection_model.h5')

# Use the loaded model to make predictions on new data
def detect_defect(image_path):
    img = load_image(image_path)  # Load the image (implement this function)
    img = preprocess_image(img)  # Preprocess the image (implement this function)

    # Make predictions using the loaded model
    prediction = loaded_model.predict(np.expand_dims(img, axis=0))

    # Check the prediction result
    if prediction[0][0] >= 0.5:  # Assuming binary classification (defect or non-defect)
        result = "Defect Detected"
    else:
        result = "No Defect Detected"

    # Display the image and the result
    plt.imshow(cv2.imread(image_path))  # Show the original image
    plt.title(result)
    plt.axis('off')  # Hide axis labels
    plt.show()

    return result


In [ ]:
# Example usage

image_path = '/home/topister/Desktop/ML/GroupAssignment1/Transfer Learning/NEU Metal Surface Defects Data/test/Crazing/Cr_1.bmp'  # Replace with your image path
detection_result = detect_defect(image_path)
print("Detection Result:", detection_result)

In [ ]:
# # Replace 'test_image.jpg' with the path to your test image
# test_image_path = '/content/drive/My Drive/ML/Transfer Learning/NEU Metal Surface Defects Data/test/Crazing/Cr_109.bmp'

# result = detect_defect(test_image_path)
# print(result)

In [ ]:
# # Replace 'test_image.jpg' with the path to your test image
# test_image_path = '/content/drive/My Drive/ML/Transfer Learning/NEU Metal Surface Defects Data/test/Crazing/Cr_108.bmp'

# result = detect_defect(test_image_path)
# print(result)

In [ ]:
# # Replace 'test_image.jpg' with the path to your test image
# test_image_path = '/content/drive/My Drive/ML/Transfer Learning/NEU Metal Surface Defects Data/test/Crazing/Cr_104.bmp'

# result = detect_defect(test_image_path)
# print(result)

In [ ]:
# # Replace 'test_image.jpg' with the path to your test image
# test_image_path = '/content/drive/My Drive/ML/Transfer Learning/NEU Metal Surface Defects Data/train/Rolled/RS_183.bmp'

# result = detect_defect(test_image_path)
# print(result)



In [ ]:
# # Replace 'test_image.jpg' with the path to your test image
# test_image_path = '/content/drive/My Drive/ML/Transfer Learning/img064.jpg'

# result = detect_defect(test_image_path)
# print(result)

